# Creating a parallel parameter sweep application from an existing serial version.

In a previous notebook assignment we created a serial parameter sweep application and ran the code from the Jupyter Notebook.  Using that code as a starting point we want to speed it up by distributing the work across multiple processing units.  Before we start there are a couple of concepts we need to consider.  Note that the subset defined below has been selected because the concepts directly affect the design of our parallel parameter sweep implementation.  

# Parallel Computing Concepts

## SPMD v MPMD

SPMD stands for "Single Program Multiple Data".  The Single Program Multiple Data process means that a single program runs on all of the processors or cores but each process works on different data, e.g. each loop of a for loop includes the same calculations but operates on different indices or inputs. 

MPMD stands for "Multiple Programs Multiple Data".  In this workflow each process has its own compute task,  and its own data. In some cases the programming language choice is different between individual tasks. An example of a MPMD workflow would be a processing pipeline where each separate task in the pipeline is on a distinct processor, or sest of processors, such as the signal processing pipeline shown below. Note that in this case the processing can be divided by function or task.  For this reason, MPMD models are seen more frequently in Signal and Image Processing and Data Analysis applications. 


## Shared Memory v Distributed Memory


In addition to understanding the different types of workflows, we need to consider the underlying architecture of the machine where the program will run.  Traditionally, machines were created with either shared or distributed memory.  More recently supercomputer architectures include "hybrid" memory, where the system is created from a number of shared memory nodes.  

**Shared memory** systems are those where all of the processors can see the memory space.  Imagine that you are part of a team that is meeting in a conference room.  During the discussion people are taking notes on the white board.  Everyone in the room can access the white board, read it, write on it and erase it. In the computer system this is equivalent to each process being able to read from and write to the same memory space. 

**Distributed memory** systems are those where each machine is a separate island with its own processing cores and memory. Let's revisit the meeting example we introduced with shared memory.  Imagine that the project team is spread geographically.  In this case they are all remote from one another so everyone works in their own office on their own white board. In this case when you need to share information with another team member you need to send a message, or make a phone call, to your teammate's office.  Similarly, when someone else needs information they need to send a message to you or call you.  Furthermore, in order for this communication to happen, each team member needs to know the office locations, or phone numbers, of all the team members and their tasks. 

Distributed memory processing splits the processing between machines (generally referred to as nodes) that have their own local memory.  Each time the application code needs to send its results to another node or requires results processed on a different node, the information must be communicated or "messaged" between the two nodes.  As in the case of the team meetings, the nodes must understand which other nodes are involved in the compute job and which parts of the job they are processing. Understanding the locations of all the processors (meeting rooms) and what is happening in each is known as the **local-global mapping**.

**Hybrid systems** In these systems, each node has many processors which can share the same memory.  To create a larger system, many shared memory nodes are combined, however, processors can only see the memory on their node, so messages must be sent between nodes. This model, the hybrid system, includes both shared and distributed memory systems. 

The analogous meeting example for hybrid memory is to split the full team meeting into a number of smaller team meetings each held in a separate  room.  All of the members of the small team can share the white board but if they need to get information to another team they have to send a message to the conference room where the other team is meeting.  Similarly, when they need information from another team, the other team needs to come to them.  As was the case with distributed computing, the communication requires that each team knows the locations of the other conference rooms and which team has been assigned to which conference room.

Consider the meeting logistics for the three different meeting examples.  In the shared whiteboard case everyone in the room was able to talk and share with everyone else.  This meeting style didn't need anything special, just the team, the room and the whiteboard. In the case where the team was geographically 

**Programming Models** Each of the models; shared memory, distributed memory and hybrid memory has a slightly different programming model. For this example we will focus on distributed memory models.  In particular pMatlab/pOctave uses the "Data Parallel Model", also know as the "Partitioned Global Address Space" or PGAS model.  The PGAS model assumes a distributed memory architecture but provides an abstraction so that the memory is presented to the user as if it were one large shared global space. In this model, the user designs the algorithm from a global perspective, working on the application as if it were serial, and the model takes care of the details, including determining how the data is partitioned, which processor holds which data and how messages need to be passed.   




## Abstraction Through Maps

Slides

# Parallel Octave

Parallel Octave uses MIT Lincoln Laboratory's pMatlab library which is built on a Single Program Multiple Data, Distributed Memory, Data Parallel Model.  As a result, minimal code modifications are required by the application programmer in order to achieve reasonable speedup.

As we work through the process of creating a parallel implementation of the parameter sweep application we will highlight the PGAS constructs and SPMD behavior.

## Introduction to the Map construct

Slides

# Serial to Parallel 

Now that we understand some of the models that we are working with it is time to review the serial code with an eye toward modifications that we need to make.  As part of the parallelization design we need to understand:

* where the independence is, this will determine how we distributed the data
* does the data need to be gathered up to be processed as a whole unit
* which data structures need to be distributed

Our goal in converting the serial code to parallel code is to parallelize the 'for' loop within the serial implementation. 


To start, let's consider the serial parameter sweep code, reviewing each code block in the serial implementation:

* Do we need to modify the sample_function?
* Do we need to modify the array sizes?

When developing code you always want to have a way to test the correctness of the results. This is especially true when parallel versions of serial code as the addition of parallel processing can render the code difficult to debug.  One way to insure that you are able to recover the serial code from the parallel version is to include a switch that turns the parallel features on and off.  pMatlab/pOctave has been designed so that calls to parallel constructs result in a 'no op', in other words no operation is performed and the function returns.  

In the next code cell, write the appropriate code for the sample_function and array sizes:

## Parallel Skeleton

When developing code you always want to have a way to test the correctness of the results. This is especially true when parallel versions of serial code as the addition of parallel processing can render the code difficult to debug.  One way to insure that you are able to recover the serial code from the parallel version is to include a switch that turns the parallel features on and off.  pMatlab/pOctave has been designed so that calls to parallel constructs result in a 'no op', in other words no operation is performed and the function returns.  

Throughout this exercise we have provided some skeleton code to get you started, but in each step we use questions to guide the parallel code development.  

In the next code cell, write the appropriate code to use the sample_function and set the array sizes:

### Code Block
```Octave
% basic parameter sweep code 
%
% Goal: parallelize the following loop:
% for ii = 1:n
%   z(ii) = f(ii,...otherArguments)
% end % for i loop

% Turn parallelism on or off
PARALLEL = 0;

%addpath('/home/gridsan/jmullen/examples/Param_Sweep')

% Set data sizes.
m = 3; % number of output arguments
n = 16; % number of independent iterations
```

Notice that we do not need to modify either the sample function or the array sizes.  This section of the code is identical to the serial version, we need to add the path to sample_function.m and set the array sizes.


## Creating the Maps

The next step is creating the map that will distribute the data.  This step requires understanding the independence within the application.  As you look at the loop that we are parallelizing, which dimension of **z** is independent?

Recall that the map construct in pMatlab/pOctave takes the following form:

dataMap = map([row distribution column distribution], distributionType, processorList)

For our application we will 
 * use all the processors that we ask for at runtime 
 * use a block distribution  

With those criteria and the knowledge of the application, create the map in the next code cell.  

### Code Block
```Octave

% Create Maps
map1 = 1;
if PARALLEL
% Distributed Map 
   map1 = map([Np 1], {}, 0:Np-1);
end

```


## Creating the distributed output matrix and getting the global indices

In the serial version of the application, we looped over m rows and called the sample_function to insert data in n columns. In the parallel case the m rows will be distributed among Np processors, where each processor only knows about its portion of the array.  Locally each processor will run Octave on a subset of the full matrix looping rows from 1 to num_local.  

Note that the sample_function hasn't changed, so we still expect the first column to hold the global row number for each local row and the 3rd column to hold a value 2.5 times the global row number.  


In the next cell, we demonstrate how to 
 * create a dmat, or distributed matrix
 
and then for each processor we demonstrate how to 
 * get the global indices (in this case the row numbers)
 * how to get a copy of the local portion of the matrix 


### Code Block
```Octave

% Create distributed array - dmat - 
% matrix is z - data output matrix
z = zeros(n,m,map1);

% Get the global indices that are assigned to my Processor ID  (Pid)
my_i_global = global_ind(z,1);

% Get a local copy of the portion of the array that is assigned to my Pid
my_z = local(z);

```

Discussion:

* Use of the map function
  - the zeros and ones functions are overloaded so that the addition of "map1" in the calling statement results in the creation of a distributed matrix
  - note that in the serial case map1 has been defined to be 1 so that the code doesn't error out during the creation of z
  
* Use of the global_ind function
  - the global_ind function returns a vector with the global indices associated with the data that was assigned to a processor.  This is a SPMD program so each processor executes the same statement but receives a different set of indices.  

* Use of the local function
   - the local function is used to get a copy of the local array
   - this call is not required but was added to optimize the local access to the array during computation
   - to copy the updated local array back to the full dmat, the put_local command is used and will be demonstrated in a few more steps

## Modifying the for loop to enable parallel processing

Now that we have created the map, the distributed matrix z, computed the global indices and gotten a copy of the local portion of z, we are ready to create our parallel loop. 

Consider the for loop in this parameter sweep.  In serial the code loops over all of the rows and calls sample_function with 3 arguments: 
* loop index - row number in the serial case
* processor ID number, pid
* my_other_arg = 2.5 \* (global row number)

In the parallel case each process needs to loop over the rows assigned to the process via the dmat and
* compute the global row number from the local index
* know its pid (assigned as part of pMatlab/pOctave initialization)
* compute my_other_arg using the global row number

In our last code snippet we computed everything we needed to create the loop.  Let's look at how we compute the 3 arguments to sample_function.  
1. Computing the loop indices
  * my_i_global holds a vector of the global row numbers assigned to a processor
  * the length of my_i_global is the length of the local matrix in the row dimension
2. Computing global row number
  * my_i_global holds a vector of the global row numbers assigned to a processor
  * use the loop index as a pointer into my_i_global to get the global row number
3. Pid is a reserved variable, initialized with pMatlab/pOctave init and each processor knows its Pid 
4. my_other_arg is straightforward once we have the global row number


A final note about indexing before we look at the code:

The goal is to view the application from a global perspective and use the pMatlab/pOctave constructs to ease the distribution, computation and collection of data. When calling the sample_function to compute and assign results to the local portion of the distributed array you need to remember to _Compute Globally, Assign Locally_.  You do this by using the local indices for assignment into arrays and global indices for calls to functions or computations.

Note that this is importatnt because Matlab allows you to extend a matrix during runtime.  If you use global indices to assign your local matrix values, Matlab will extend the matrix to match the indices you are using. When you complete the for loop, the local matrix will not fit back into the distributed array and the code will fail with an error.

Let's look at the code.

### Code Block
```Octave

% Loop over the local indices, call sample_function and compute results for local portion of matrix
for i_local = 1:length(my_i_global)
   % use local index as pointer to my_i_global vector to get global index
   i_global = my_i_global(i_local);

   % calculate my_other_argument
   my_other_arg = 2.5*i_global;
   
   % call sample function with global index, pid, my_other_arg and store result in row of local z matrix
   my_z(i_local, :) = sample_function(i_global, pid, my_other_arg);
end

```


## Gathering the data for display

In order to produce a final result equivalent to our serial version, we need to gather the data into a single m x n matrix and display the results.  

To gather the data we use a gather, or agg, function that collects the data from each processor onto the leader processor, pid = 0.  Prior to agging the data we copy the local data back into the global dmat.  

Note that the final result of the agg command is a vanilla matlab/octave matrix on the leader processor, pid 0, and partial results on the remaining processes.  

The code looks like:

### Code Block
```Octave

% Store the local portion of z (my_z) in the distributed matrix, z
z = put_local(z, my_z);

% Gather the data to the leader process using the agg function
z_final = agg(z);

% Declare success and display the results
disp('SUCCESS');
disp(z_final);

```

# Running a pMatlab/pOctave Codes

Parallel jobs require more setup than serial jobs because the system needs to know how many processors to allocate to a job and in some cases which type of processors.

For pMatlab/pOctave jobs we use the command 
`eval(pRUN(argument1, argument2, argument3))`
where the arguments are:
* `argument1` = the name of the Octave script, without the extension, in quotes because it is a string. For example, 'myfile'.
* `argument2` = the number of processors to use when running the job
* `argument3` = the processors to use when running the job.  When running on TXE1 the argument is 'grid'.


## Exercise

Try running the parallel paramenter sweep code on 1,2, and 4 processors and collect the timing information.  The timing as implemented is very crude - how could it be improved?
* Open a terminal window
* Start up MATLAB `matlab -nodisplay -singleCompThread`
* At the Octave/MATLAB command line type:
`eval(pRUN('param_sweep_parallel_v2', #processors, 'grid'))`


# Best Practices for Adding Parallelism

**One tenet of good software engineering is that programs should not be run on full scale inputs immediately.  Rather, programs should initially be run on a small test problem to verify functionality and to validate against known results.**

The following is the recommended procedure for adding parallelism to a pMatlab application.  This procedure gradually adds complexity to running the application.

1.	**Run with 1 processor interactively (using LLsub -i) with the pMatlab library disabled.**  
 * This tests the basic serial functionality of the code.
 
2.	**Run with 1 processor interactively with pMatlab enabled.**  
 * Tests that the pMatlab library has not broken the basic functionality of the code.
 
3.	**Run with 2 processors on multiple machines.**
 * Test that the program works with network communication.
 
4.	**Run with 4 processors on multiple machines.**

5.	**Increase the number of processors, as desired.**

_Note that "More is not always better" because there is a limit to the scalability of your application code. Finding the limit usually requires that you try varying numbers of processors until you see "speed-down", i.e. you reach a point where it takes longer with more processors._  

# References

For additional information on pMatlab/pOctave see: https://www.mit.edu/~kepner/pMatlab/

For additional information on parallel programming models see: https://computing.llnl.gov/tutorials/parallel_comp/